In [1]:
import os
import pandas as pd
import numpy as np
import collections


수비점수 = 수비율 * 수비이닝 * 포지션 가중치
포지션 가중치
- 내야수
  = 1루수 2
  = 3루수 4
  = 2루수 6
  = 유격수 7
- 외야수
  = 좌익수 2
  = 우익수 2.5
  = 중견수 3.5
  
FPCT (수비율or 수비성공률) - (자살+보살) / (자살+보살+실책) 

#### 내야수

In [2]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율')
dfn_1 = pd.read_csv('2019_전체.csv')
dfn_1.index = dfn_1['선수명']
del dfn_1['선수명']
infielders = ['1루수', '2루수', '3루수', '유격수']
dfn_in = dfn_1[dfn_1['POS'].isin(infielders)]
dfn_in = dfn_in.drop(columns = ['G', 'GS', 'PKO', 'PO', 'A', 'DP', 'PB', 'SB', 'CS', 'CS%'])
dfn_in = dfn_in.replace('-', '0')
dfn_in

,팀명,POS,IP,E,FPCT
선수명,,,,,
강경학,한화,2루수,51.00,1,0.971
강경학,한화,1루수,69.00,0,1.000
강경학,한화,유격수,153.00,3,0.965
강로한,롯데,3루수,183.33,3,0.950
강로한,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000
황재균,KT,유격수,54.00,1,0.968
황재균,KT,3루수,964.00,10,0.970


In [4]:
## 이닝 수 integer 로 바꾸고 분수 없애기
IP_list = []
for f in dfn_in['IP'] :
    if '1/3' in f :
        IP_list.append(int(f.replace(' 1/3', '33'))/100)
    elif '2/3' in f :
        IP_list.append(int(f.replace(' 2/3', '66'))/100)
    else :
        IP_list.append(int(f))
dfn_in['IP'] = IP_list
dfn_in

TypeError: argument of type 'float' is not iterable

In [5]:
## 수비율 integert로 바꾸고 소수점 세자리까지 표기
dfn_in['FPCT'] = [int(b.replace('.', '')) / 1000  for b in dfn_in['FPCT']]
dfn_in

,팀명,POS,IP,E,FPCT
선수명,,,,,
강경학,한화,2루수,51.00,1,0.971
강경학,한화,1루수,69.00,0,1.000
강경학,한화,유격수,153.00,3,0.965
강로한,롯데,3루수,183.33,3,0.950
강로한,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000
황재균,KT,유격수,54.00,1,0.968
황재균,KT,3루수,964.00,10,0.970


#### 내가 임의로 가중치 반영

In [6]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_in)) :
    if dfn_in.iloc[a, 1] == '1루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 2, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '3루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 4, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '2루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 6, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '유격수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 7, 3)
        D_score_list.append(D_score)
dfn_in['D_score'] = D_score_list
dfn_in

,팀명,POS,IP,E,FPCT,D_score
선수명,,,,,,
강경학,한화,2루수,51.00,1,0.971,297.126
강경학,한화,1루수,69.00,0,1.000,138.000
강경학,한화,유격수,153.00,3,0.965,1033.515
강로한,롯데,3루수,183.33,3,0.950,696.654
강로한,롯데,유격수,261.67,12,0.911,1668.670
...,...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000,17.340
황재균,KT,유격수,54.00,1,0.968,365.904
황재균,KT,3루수,964.00,10,0.970,3740.320


#### kWAR 포지션 조정 가중치 반영

In [7]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_in)) :
    if dfn_in.iloc[a, 1] == '1루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 0, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '3루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '2루수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_in.iloc[a, 1] == '유격수' :
        D_score = round(dfn_in.iloc[a, 4] * dfn_in.iloc[a, 2] * 0.8, 3)
        D_score_list.append(D_score)
dfn_in['D_score'] = D_score_list
dfn_in

,팀명,POS,IP,E,FPCT,D_score
선수명,,,,,,
강경학,한화,2루수,51.00,1,0.971,29.713
강경학,한화,1루수,69.00,0,1.000,0.000
강경학,한화,유격수,153.00,3,0.965,118.116
강로한,롯데,3루수,183.33,3,0.950,104.498
강로한,롯데,유격수,261.67,12,0.911,190.705
...,...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000,0.000
황재균,KT,유격수,54.00,1,0.968,41.818
황재균,KT,3루수,964.00,10,0.970,561.048


In [8]:
dfn_in_uniq = dfn_in.reset_index()
dfn_in_uniq = dfn_in_uniq.append({'선수명' : 0},  ignore_index = True) ## 루프 돌릴 때 마지막 row 하나 더 필요해서 추가
dfn_in_uniq = dfn_in_uniq.drop(columns = ['팀명', 'POS'])
dfn_in_uniq

,선수명,IP,E,FPCT,D_score
0,강경학,51.00,1.0,0.971,29.713
1,강경학,69.00,0.0,1.000,0.000
2,강경학,153.00,3.0,0.965,118.116
3,강로한,183.33,3.0,0.950,104.498
4,강로한,261.67,12.0,0.911,190.705
...,...,...,...,...,...
301,황재균,54.00,1.0,0.968,41.818
302,황재균,964.00,10.0,0.970,561.048
303,황진수,1.00,0.0,0.000,0.000
304,황진수,9.33,0.0,1.000,5.598


In [9]:
## 선수 한 명의 기록 합친 값이 들어갈 df 생성
dfn_in_uniqs = pd.DataFrame(columns = dfn_in_uniq.columns)
dfn_in_uniqs['선수명'] = dfn_in_uniq['선수명'].unique()
dfn_in_uniqs

,선수명,IP,E,FPCT,D_score
0,강경학,NaN,NaN,NaN,NaN
1,강로한,NaN,NaN,NaN,NaN
2,강민국,NaN,NaN,NaN,NaN
3,강백호,NaN,NaN,NaN,NaN
4,강승호,NaN,NaN,NaN,NaN
...,...,...,...,...,...
151,황대인,NaN,NaN,NaN,NaN
152,황윤호,NaN,NaN,NaN,NaN
153,황재균,NaN,NaN,NaN,NaN
154,황진수,NaN,NaN,NaN,NaN


In [10]:
## 선수 한 명의 1년치 기록 합치기
ID_1 = '강경학'
row = 0
for d in range(1, 5) :
    c = 0
    row = 0
    for a in range(0, len(dfn_in_uniq)) :
        ID = dfn_in_uniq.iloc[a, 0]
        if ID == ID_1 :
            c += dfn_in_uniq.iloc[a, d]

        else :
            dfn_in_uniqs.iloc[row, d] = c
            row += 1
            c = 0
            c += dfn_in_uniq.iloc[a, d]
            ID_1 = dfn_in_uniq.iloc[a, 0]

In [11]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 첫 번째 칼럼도 하나씩 밀어줌
IP_list = dfn_in_uniqs['IP'].to_list()
IP_list.insert(0, 0)
IP_list = IP_list[:-1]
dfn_in_uniqs['IP'] = IP_list

In [12]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 두 번째 칼럼도 하나씩 밀어줌
ID_list = dfn_in_uniqs['선수명'].to_list()
ID_list.insert(0, 0)
ID_list = ID_list[:-1]
dfn_in_uniqs['선수명'] = ID_list

In [13]:
## 그리고 나서 첫 번째 쓸모없는 row 삭제
dfn_in_uniqs = dfn_in_uniqs.drop([0])
dfn_in_uniqs = dfn_in_uniqs.reset_index(drop = True)
dfn_in_uniqs

,선수명,IP,E,FPCT,D_score
0,강경학,273.00,4.0,2.936,147.829
1,강로한,709.67,20.0,2.834,449.717
2,강민국,245.67,9.0,2.734,171.306
3,강백호,3.00,0.0,1.0,0.0
4,강승호,94.00,1.0,1.929,59.353
...,...,...,...,...,...
150,홍재호,49.00,1.0,1.96,29.792
151,황대인,48.00,0.0,2.0,1.2
152,황윤호,297.67,6.0,2.9,177.799
153,황재균,1026.67,11.0,2.938,602.866


In [14]:
## 수비위치 몇개까지 멀티를 했는지 count
counts = collections.Counter(dfn_in_uniq['선수명'])
df_cnt = pd.DataFrame()
df_cnt['선수명'] = counts.keys()
df_cnt['포지션 수'] = counts.values()
df_cnt = df_cnt[:-1]
df_cnt

,선수명,포지션 수
0,강경학,3
1,강로한,3
2,강민국,3
3,강백호,1
4,강승호,3
...,...,...
150,홍재호,2
151,황대인,2
152,황윤호,3
153,황재균,3


In [15]:
## df에 추가
dfn_in_uniqs['포지션 수'] = df_cnt['포지션 수']
dfn_in_uniqs

,선수명,IP,E,FPCT,D_score,포지션 수
0,강경학,273.00,4.0,2.936,147.829,3
1,강로한,709.67,20.0,2.834,449.717,3
2,강민국,245.67,9.0,2.734,171.306,3
3,강백호,3.00,0.0,1.0,0.0,1
4,강승호,94.00,1.0,1.929,59.353,3
...,...,...,...,...,...,...
150,홍재호,49.00,1.0,1.96,29.792,2
151,황대인,48.00,0.0,2.0,1.2,2
152,황윤호,297.67,6.0,2.9,177.799,3
153,황재균,1026.67,11.0,2.938,602.866,3


In [16]:
## 수비율 평균
dfn_in_uniqs['FPCT'] = [i/dfn_in_uniqs.iloc[j, 5] for j, i in enumerate(dfn_in_uniqs['FPCT'])]
dfn_in_uniqs

,선수명,IP,E,FPCT,D_score,포지션 수
0,강경학,273.00,4.0,0.978667,147.829,3
1,강로한,709.67,20.0,0.944667,449.717,3
2,강민국,245.67,9.0,0.911333,171.306,3
3,강백호,3.00,0.0,1.000000,0.0,1
4,강승호,94.00,1.0,0.643000,59.353,3
...,...,...,...,...,...,...
150,홍재호,49.00,1.0,0.980000,29.792,2
151,황대인,48.00,0.0,1.000000,1.2,2
152,황윤호,297.67,6.0,0.966667,177.799,3
153,황재균,1026.67,11.0,0.979333,602.866,3


In [232]:
# ## D_score 다시 계산
# dfn_in_uniqs['D_score'] = 

In [17]:
dfn_in_uniqs.index = dfn_in_uniqs['선수명']
#del dfn_in_uniqs['선수명'], dfn_in_uniqs['FPCT']
dfn_in_uniqs

,선수명,IP,E,FPCT,D_score,포지션 수
선수명,,,,,,
강경학,강경학,273.00,4.0,0.978667,147.829,3
강로한,강로한,709.67,20.0,0.944667,449.717,3
강민국,강민국,245.67,9.0,0.911333,171.306,3
강백호,강백호,3.00,0.0,1.000000,0.0,1
강승호,강승호,94.00,1.0,0.643000,59.353,3
...,...,...,...,...,...,...
홍재호,홍재호,49.00,1.0,0.980000,29.792,2
황대인,황대인,48.00,0.0,1.000000,1.2,2
황윤호,황윤호,297.67,6.0,0.966667,177.799,3


In [234]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율\전처리')
dfn_in_uniqs.to_csv('2019_내야수_수비점수_kWAR.csv', encoding = 'cp949')